# Resnet18, 34 구현
참조: https://youtu.be/671BsKl8d0E.   
https://github.com/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb

In [16]:
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import torchvision
import torchvision.transforms as transforms

## 모델 쌓기

In [54]:
class BasicBlock(nn.Module):
  def __init__(self, in_channels, out_channels, stride = 1):
    super(BasicBlock, self).__init__()

    # 첫 번째 convolution layer
    # input과 output의 차원 다를 때: stride로 너비, 높이 조절
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                           stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)

    # 두 번째 convolution layer
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                           stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)

    # shortcut connections
    # nn.Sequential(): 모듈들을 인수로 받아 순서대로 정렬해 둔 뒤 입력값이 들어오면 순서대로 모듈 실행하여 결과값 Return 
    self.shortcut = nn.Sequential()
    if stride != 1: #stride가 1이 아님. 차원이 달라 identity mapping을 수행하지 않는 경우
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
          nn.BatchNorm2d(out_channels)
      )

  def forward(self, x):
    # F.relu: pytorch의 내장 relu 함수
    # convolution layer 1 -> bn -> relu -> convolution layer 2 -> bn -> shortcut -> relu
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out = out + self.shortcut(x)
    out = F.relu(out)
    return out

In [55]:
class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_channels = 64

    # 3x3 필터 64개
    # 하나의 convolution layer로 dimension 바꿔 줌
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)

    # basic block 쌓기
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512, num_classes)

  def _make_layer(self, block, out_channels, num_blocks, stride):
    # strides: 너비, 높이 조절할 수 있는 1이 아닌 stride 값 + block 개수 만큼의 1
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_channels, out_channels, stride))
      self.in_channels = out_channels # 다음 레이어의 input 채널 수 변경
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    # FC
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

In [57]:
class Model:
  def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

  def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

## 데이터셋 다운로드

In [58]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(), # data augmentation 
    transforms.ToTensor(),])

transform_test = transforms.Compose([
    transforms.ToTensor(),])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


## 학습 환경설정

In [62]:
device = 'cuda'

net = Model.ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1 #0.1로 시작해서 점차 줄여나갈 예정
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

## 학습 과정 설계

In [63]:
def adjust_lr(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def train(epoch):
    print('\n[ Train %d]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

def test(epoch):
    print('\n[ Test %d]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

## 학습

In [64]:
for epoch in range(0, 20):
    adjust_lr(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train 0]


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(



Current batch: 0
Current benign train accuracy: 0.0859375
Current benign train loss: 2.4415760040283203

Current batch: 100
Current benign train accuracy: 0.1640625
Current benign train loss: 2.099649667739868

Current batch: 200
Current benign train accuracy: 0.25
Current benign train loss: 1.956758737564087

Current batch: 300
Current benign train accuracy: 0.3125
Current benign train loss: 1.8152332305908203

Total benign train accuarcy: 26.366
Total benign train loss: 811.7784942388535

[ Test 0]

Test accuarcy: 31.8
Test average loss: 0.01968365511894226
Model Saved!

[ Train 1]

Current batch: 0
Current benign train accuracy: 0.390625
Current benign train loss: 1.585494041442871

Current batch: 100
Current benign train accuracy: 0.421875
Current benign train loss: 1.6850682497024536

Current batch: 200
Current benign train accuracy: 0.4609375
Current benign train loss: 1.4859141111373901

Current batch: 300
Current benign train accuracy: 0.53125
Current benign train loss: 1.2798